In [ ]:
"""
Copyright 2021 Eric Silk

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and
associated documentation files (the "Software"), to deal in the Software without restriction,
including without limitation the rights to use, copy, modify, merge, publish, distribute,
sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT
NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM,
DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT
OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""
import os
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import multiprocessing as mp

DATA_DIR = "./levenberg_data/"
PICKLE_NAME = "levenberg_rslts_df.pkl"

def read_json_to_df(fname):
    try:
        with open(fname) as f:
            tmp_df = pd.json_normalize(json.load(f))
            tmp_df.drop("all_losses", 1, inplace=True)
            tmp_df.drop("all_lambdas", 1, inplace=True)
            return tmp_df
    except Exception:
        return None

if not os.path.exists(PICKLE_NAME):
    print("Creating dataframe!")
    assert(os.path.exists(DATA_DIR))
    f_list = glob.glob(os.path.join(DATA_DIR, "TEST_*.json"))
    print(len(f_list))
    with mp.Pool() as p:
        dframes = p.map(read_json_to_df, f_list)

    dframes = [i for i in dframes if i is not None]

    monolith = pd.concat(dframes)

    monolith.to_pickle(PICKLE_NAME)
else:
    print("Reading dataframe!")
    monolith = pd.read_pickle("levenberg_rslts_df.pkl")
#monolith.rename(columns=lambda x: x.replace(".", "_"), inplace=True)
monolith.keys()

In [ ]:
monolith.head()


In [ ]:
num_hidden = 75
cifar = monolith.loc[monolith["specs.dataset"] == "cifar10"]
MLP = monolith.loc[monolith["specs.dataset"] == "mnist"]
MLP = MLP.loc[MLP["specs.hidden"] == num_hidden]
mlp_sgd = MLP.loc[MLP["specs.opt"] == "sgd"]
mlp_levenberg = MLP.loc[MLP["specs.opt"] == "levenberg"]
print(len(mlp_sgd))
print(len(mlp_levenberg))
print(len(cifar))

In [ ]:
def max_acc_lambda(row):
    try:
        max_ = max(row["test_accuracy_list"])
    except ValueError:
        max_ = 0.0
    return max_

def final_acc_lambda(row):
    try:
        last = row["test_accuracy_list"][-1]
    except ValueError:
        last = 0.0
    return last

cumulative_time_lambda = lambda row: np.cumsum(row["time"])

def total_time_lambda(row):
    try:
        max_ = row["time"][-1]
    except ValueError:
        max_ = 0.0
    return max_

def get_summary_vals(df):
    df["top_test_acc"] = df.apply(max_acc_lambda, axis=1)
    df["final_test_acc"] = df.apply(final_acc_lambda, axis=1)
    df["training_timestamps"] = df.apply(cumulative_time_lambda, axis=1)
    df["total_training_time"] = df.apply(total_time_lambda, axis=1)

    return df

mlp_sgd = get_summary_vals(mlp_sgd)
mlp_levenberg = get_summary_vals(mlp_levenberg)
cifar = get_summary_vals(cifar)

In [ ]:
def get_n_best_acc(full_df, n):
    full_df.sort_values(["top_test_acc"], ascending=False, inplace=True)
    return full_df.head(n)

def get_n_best_final_acc(full_df, n):
    full_df.sort_values(["final_test_acc"], ascending=False, inplace=True)
    return full_df.head(n)

best_acc_mlp_sgd = get_n_best_acc(mlp_sgd, 5)
best_final_acc_mlp_sgd = get_n_best_final_acc(mlp_sgd, 5)

best_acc_mlp_levenberg = get_n_best_acc(mlp_levenberg, 5)
best_final_acc_mlp_levenberg = get_n_best_final_acc(mlp_levenberg, 5)

best_acc_cifar = get_n_best_acc(cifar, 5)
best_final_acc_cifar = get_n_best_final_acc(cifar, 5)

In [ ]:
def plot_rows(df: pd.DataFrame, best_column):
    # vs. Epoch
    plt.figure(figsize=(16, 9))
    for i, row in enumerate(df.itertuples()):
        plt.plot(row.test_accuracy_list)
    plt.xlabel("Epoch")
    plt.ylabel("Percent correct")
    best = df.head(1)
    cols_of_interest = [
        "specs.opt",
        "specs.learning_rate",
        "specs.momentum",
        "specs.batch_size_train",
        "specs.hidden",
        "specs.max_newton",
        "specs.lambda0",
        "specs.nu"
    ]
    specs = [col for col in best.columns if col in cols_of_interest]
    top_acc = best[best_column][0]
    plt.title("Best: {}%\n{}".format(top_acc, best[specs].to_dict()))

    # vs. Time
    plt.figure(figsize=(16, 9))
    for i, row in enumerate(df.itertuples()):
        plt.plot(row.training_timestamps, row.test_accuracy_list)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Percent correct")
    best = df.head(1)
    cols_of_interest = [
        "specs.opt",
        "specs.learning_rate",
        "specs.momentum",
        "specs.batch_size_train",
        "specs.hidden",
        "specs.max_newton",
        "specs.lambda0",
        "specs.nu"
    ]
    specs = [col for col in best.columns if col in cols_of_interest]
    top_acc = best[best_column][0]
    plt.title("Best: {}%\n{}".format(top_acc, best[specs].to_dict()))

plot_rows(best_acc_mlp_sgd, "top_test_acc")
plot_rows(best_final_acc_mlp_sgd, "final_test_acc")

In [ ]:
plot_rows(best_acc_mlp_levenberg, "top_test_acc")
plot_rows(best_final_acc_mlp_levenberg, "final_test_acc")

In [ ]:
plot_rows(best_acc_cifar, "top_test_acc")
plot_rows(best_final_acc_cifar, "final_test_acc")

In [ ]:
print(monolith.columns)
def param_vs_final_acc(df, param_name):
    series = df[param_name]
    unique_vals = series.unique()
    accs = []
    for val in unique_vals:
        tmp = df.loc[df[param_name] == val]
        accs = tmp["final_test_acc"]
    plt.figure(figsize=(16, 9))
    for val, acc in zip(unique_vals, accs):

def param_vs_top_acc(df, param_name):
    plt.figure(figsize=(16, 9))
    plt.scatter(df[param_name], df["top_test_acc"])
    plt.title(f"Top test accuracy vs. {param_name}")

param_vs_final_acc(monolith, "specs.learning_rate")
param_vs_top_acc(monolith, "specs.learning_rate")
param_vs_final_acc(monolith, "specs.nu")
param_vs_top_acc(monolith, "specs.nu")
